In [1]:
!pip install --user xlrd

In [2]:
# Setup constants if any
# FUNDS ID
FUNDS_ID_LIST = ['I-CJF','I-MG1','I-SQGFS']



In [3]:
import pandas as pd
from io import BytesIO
import requests
import json
import xlrd 

from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime
from dateutil.parser import parse

from ingest.Connectors import Connectors

In [4]:
# The code was removed by DSX for sharing.

In [5]:
# The code was removed by DSX for sharing.

In [6]:
fundNavDF = pd.read_excel(getFileFromObjectStorage('MizuhoPOC', 'Fund_NAV_MAI_DailyPLfiles_20170731.xlsm'), 
                              header=[0], skipinitialspace=True, 
                              skiprows=8, 
                              usecols=['Portfolio', 'Base CCY','NAV (Base CCY)']).dropna(axis=[0,1])

fundNavRenamedDF = fundNavDF.rename(index=str, columns={"Portfolio":"FUND_ID","Base CCY": "BASE_CURR","NAV (Base CCY)": "NAV_BASE_CCY"})

fundNavRenamedDF[['NAV_BASE_CCY']] = fundNavRenamedDF[['NAV_BASE_CCY']].astype(float)

asOfDate = pd.to_datetime('today').strftime("%Y-%m-%d")
print asOfDate

fundNavRenamedDF.dtypes



2017-10-09


FUND_ID          object
BASE_CURR        object
NAV_BASE_CCY    float64
dtype: object

In [7]:
spark = SparkSession.builder.getOrCreate()  

def build_schema():
    """Build and return a schema to use for the sample data."""
    schema = StructType(
        [            
            StructField("FUND_ID",  StringType(), False),
            StructField("BASE_CURR", StringType(), True),
            StructField("NAV_BASE_CCY", DoubleType(), True)
        ]
    )
    return schema


fundNavDFSparkDF = spark.createDataFrame(fundNavRenamedDF, schema=build_schema()) \
                                .withColumn("AS_OF_DATE", lit(asOfDate).cast("date"))\
                                .select(col("FUND_ID"),col("AS_OF_DATE"), col("BASE_CURR"), col("NAV_BASE_CCY").cast(DecimalType(20,10)))


fundNavDFSparkDF.printSchema()
fundNavDFSparkDF.head(1)


root
 |-- FUND_ID: string (nullable = false)
 |-- AS_OF_DATE: date (nullable = true)
 |-- BASE_CURR: string (nullable = true)
 |-- NAV_BASE_CCY: decimal(20,10) (nullable = true)



[Row(FUND_ID=u'Mizuho Alternative Investments, LLC', AS_OF_DATE=datetime.date(2017, 10, 9), BASE_CURR=u'USD', NAV_BASE_CCY=Decimal('989399546.1000000000'))]

In [9]:
dashDBloadOptions = { 
                    Connectors.DASHDB.HOST              : dashCredentials["host"],
                    Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                    Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                    Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                    Connectors.DASHDB.SOURCE_TABLE_NAME : dashCredentials["REF_FUND_TABLE"],
}

refFundDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
refFundDF.printSchema()
refFundDF.show(1)

root
 |-- ID: string (nullable = false)

+-----+
|   ID|
+-----+
|I-CJF|
+-----+
only showing top 1 row



In [10]:
fundNavDFJoinSparkDF = fundNavDFSparkDF.join(refFundDF, 
                                               fundNavDFSparkDF.FUND_ID == refFundDF.ID, "inner")\
                                        .select(
                                            refFundDF.ID.alias("FUND_ID"),
                                            fundNavDFSparkDF.BASE_CURR,
                                            fundNavDFSparkDF.NAV_BASE_CCY,
                                            fundNavDFSparkDF.AS_OF_DATE,                                                                                              
                                            ).orderBy("FUND_ID")

fundNavDFJoinSparkDF.show(19)

+---------+---------+--------------------+----------+
|  FUND_ID|BASE_CURR|        NAV_BASE_CCY|AS_OF_DATE|
+---------+---------+--------------------+----------+
|   I-AQUA|      USD| 29577088.9600000000|2017-10-09|
|    I-CGF|      USD|  -388666.1400000000|2017-10-09|
|    I-CJF|      USD| 56683885.3700000000|2017-10-09|
|    I-MG1|      USD|106622170.6200000000|2017-10-09|
|  I-RLLAF|      USD| 15010831.0900000000|2017-10-09|
|  I-SQGFS|      USD| 50374707.5700000000|2017-10-09|
| I-SQGFSO|      USD|  6457979.2300000000|2017-10-09|
|I-WEUBEAR|      JPY|936984347.3400000000|2017-10-09|
|I-WEUBULL|      JPY|                null|2017-10-09|
|I-WGEBEAR|      JPY|806685713.8000000000|2017-10-09|
|I-WGEBULL|      JPY|1081921808.370000...|2017-10-09|
|   I-WGEF|      USD| 10003797.5000000000|2017-10-09|
|I-WMOBEAR|      JPY|1007773488.400000...|2017-10-09|
|I-WMOBULL|      JPY|1029205326.220000...|2017-10-09|
|I-WUEBEAR|      JPY|1002523582.030000...|2017-10-09|
|I-WUEBULL|      JPY|4894914

In [162]:


# Connection to Dash DB for writing the data
dashdbsaveoption = {
                     Connectors.DASHDB.HOST              : dashCredentials["host"],
                     Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                     Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                     Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                     Connectors.DASHDB.TARGET_TABLE_NAME : dashCredentials["tableName"],
                     Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' 
}

fundNavDashDBDF = fundNavDFJoinSparkDF.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()
